# LLM Recommender

## 1. Create a workpace in your workspace group

## 2. Create a database in the workspace

## 3. Install and import required libraries

In [ ]:
%pip install singlestoredb openai tiktoken beautifulsoup4 pandas python-dotenv Markdown praw PyGithub tweepy --quiet

import re
import json
import openai
import tiktoken
import json
import requests
import pandas as pd
import singlestoredb as s2
from bs4 import BeautifulSoup
from markdown import markdown
from datetime import datetime
from time import time

from github import Github
from github import Auth
import tweepy
import praw

## 4. Set variables

### 4.1. Set the app common variables

In [ ]:
MODELS_LIMIT = 1000
MODELS_TABLE_NAME = 'models'
MODEL_READMES_TABLE_NAME = 'model_readmes'
MODEL_TWITTER_POSTS_TABLE_NAME = 'model_twitter_posts'
MODEL_REDDIT_POSTS_TABLE_NAME = 'model_reddit_posts'
MODEL_GITHUB_REPOS_TABLE_NAME = 'model_github_repos'
LEADERBOARD_DATASET_URL = 'https://llm-recommender.vercel.app/datasets/leaderboard.json'
TOKENS_LIMIT = 2047
TOKENS_TRASHHOLD_LIMIT = TOKENS_LIMIT - 128

### 4.2. Set the OpenAI variables
1. [Open the OpenAI API keys page](https://platform.openai.com/api-keys)
2. Create a new key
3. Copy the key and paste it into the `OPENAI_API_KEY` variable

In [ ]:
OPENAI_API_KEY = '<YOUR_OPENAI_API_TOKEN>'

### 4.3. Set the HuggingFace variables
1. [Open the HuggingFace Access Tokens page](https://huggingface.co/settings/tokens)
2. Create a new token
3. Copy the key and paste it into the `HF_TOKEN` variable

In [ ]:
HF_TOKEN = ''

### 4.4. Set the Reddit variables

1. [Open the Reddit Apps page](https://www.reddit.com/prefs/apps)
2. Click on the `Create another app` button
3. Fill the form:
    - `name` - name the app as you wish
    - `redirect uri` - enter any http address, for example: http://localhost:4000
4. Click on the `Create app` button
5. Copy the `personal use script` value (it is located right below the app name) and paste it into the `REDDIT_CLIENT_ID` variable
6. Copy the `secret` value and paste it into the `REDDIT_CLIENT_SECRET` variable
7. Enter values into the remaining variables

In [ ]:
REDDIT_USERNAME = ''
REDDIT_PASSWORD = ''
REDDIT_CLIENT_ID = ''
REDDIT_CLIENT_SECRET = ''
REDDIT_USER_AGENT = 'llm_recommender_1.0'

### 4.5. Set the Twitter variables
1. [Open the Twitter Developer Projects & Apps page](https://developer.twitter.com/en/portal/projects-and-apps)
2. Add a new app
3. Fill the form
4. Generate a Bearer Token and paste it into the `TWITTER_BEARER_TOKEN` variable

In [ ]:
TWITTER_BEARER_TOKEN = ''

### 4.6. Set the GitHub variables
1. [Open the Register new GitHub App page](https://github.com/settings/apps/new)
2. Fill the form
3. Get an access token and paste it into the `GITHUB_ACCESS_TOKEN` variable

In [ ]:
GITHUB_ACCESS_TOKEN = ''

## 5. Create a database connection and functions

- `connection` - database connection to execute queries
- `create_tables` - function that creates empty tables in the database
- `drop_table` - helper function to drop a table
- `get_models` - helper function to get models from the models table
- `db_get_last_created_at` - helper function to get last `created_at` value from a table

The `create_tables` creates the following tables:
- `models_table` - table with all models data from the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)
- `readmes_table` - table with model readme texts from the HugginFace model pages (used in semantic search)
- `twitter_posts` - table with tweets related to models (used in semantic search)
- `reddit_posts` - table with Reddit posts related to models (used in semantic search)
- `github_repos` - table with GitHub readme texts related to models (used in semantic search)

In [ ]:
connection = s2.connect(connection_url)


def create_tables():
    def create_models_table():
        with connection.cursor() as cursor:
            cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {MODELS_TABLE_NAME} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(512) NOT NULL,
                    author VARCHAR(512) NOT NULL,
                    repo_id VARCHAR(1024) NOT NULL,
                    score DECIMAL(5, 2) NOT NULL,
                    arc DECIMAL(5, 2) NOT NULL,
                    hellaswag DECIMAL(5, 2) NOT NULL,
                    mmlu DECIMAL(5, 2) NOT NULL,
                    truthfulqa DECIMAL(5, 2) NOT NULL,
                    winogrande DECIMAL(5, 2) NOT NULL,
                    gsm8k DECIMAL(5, 2) NOT NULL,
                    link VARCHAR(255) NOT NULL,
                    downloads INT,
                    likes INT,
                    still_on_hub BOOLEAN NOT NULL,
                    created_at TIMESTAMP,
                    embedding BLOB
                )
            ''')

    def create_model_readmes_table():
        with connection.cursor() as cursor:
            cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {MODEL_READMES_TABLE_NAME} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    model_repo_id VARCHAR(512),
                    text LONGTEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
                    clean_text LONGTEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
                    created_at TIMESTAMP,
                    embedding BLOB
                )
            ''')

    def create_model_twitter_posts_table():
        with connection.cursor() as cursor:
            cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {MODEL_TWITTER_POSTS_TABLE_NAME} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    model_repo_id VARCHAR(512),
                    post_id VARCHAR(256),
                    clean_text LONGTEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
                    created_at TIMESTAMP,
                    embedding BLOB
                )
            ''')

    def create_model_reddit_posts_table():
        with connection.cursor() as cursor:
            cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {MODEL_REDDIT_POSTS_TABLE_NAME} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    model_repo_id VARCHAR(512),
                    post_id VARCHAR(256),
                    title VARCHAR(512) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
                    clean_text LONGTEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
                    link VARCHAR(256),
                    created_at TIMESTAMP,
                    embedding BLOB
                )
            ''')

    def create_model_github_repos_table():
        with connection.cursor() as cursor:
            cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {MODEL_GITHUB_REPOS_TABLE_NAME} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    model_repo_id VARCHAR(512),
                    repo_id INT,
                    name VARCHAR(512) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
                    description TEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
                    clean_text LONGTEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
                    link VARCHAR(256),
                    created_at TIMESTAMP,
                    embedding BLOB
                )
            ''')

    create_models_table()
    create_model_readmes_table()
    create_model_twitter_posts_table()
    create_model_reddit_posts_table()
    create_model_github_repos_table()


def drop_table(table_name: str):
    with connection.cursor() as cursor:
        cursor.execute(f'DROP TABLE IF EXISTS {table_name}')


def get_models(select='*', query='', as_dict=True):
    with connection.cursor() as cursor:
        _query = f'SELECT {select} FROM {MODELS_TABLE_NAME}'

        if query:
            _query += f' {query}'

        cursor.execute(_query)

        if as_dict:
            columns = [desc[0] for desc in cursor.description]
            return [dict(zip(columns, row)) for row in cursor.fetchall()]

        return cursor.fetchall()


def db_get_last_created_at(table, repo_id, to_string=False):
    with connection.cursor() as cursor:
        cursor.execute(f"""
            SELECT UNIX_TIMESTAMP(created_at) FROM {table}
            WHERE model_repo_id = '{repo_id}'
            ORDER BY created_at DESC
            LIMIT 1
        """)

        rows = cursor.fetchone()
        created_at = float(rows[0]) if rows and rows[0] else None

        if (created_at and to_string):
            created_at = datetime.fromtimestamp(created_at)
            created_at = created_at.strftime('%Y-%m-%dT%H:%M:%SZ')

        return created_at

## 6. Set the AI API key and create AI functions

### 6.1. Assing the `openapi_key`

In [ ]:
openapi_key = OPENAI_API_KEY

### 6.2. Create the `count_tokens` function
This function used to count text tokens. It is used when a long text needs to be broken into chunks.

In [ ]:
def count_tokens(text: str):
    enc = tiktoken.get_encoding('cl100k_base')
    return len(enc.encode(text, disallowed_special={}))

### 6.3. Create the `create_embedding` function
This function used to create an embedding based on an input.

In [ ]:
def create_embedding(input):
    try:
        data = openai.create(input=input, model='text-embedding-ada-002').data
        return data[0].embedding
    except Exception as e:
        print(e)
        return [[]]

## 7. Create utils

### 7.1 Create the `JSONEncoder` class
This class helps to convert datetime into the right JSON format 

In [ ]:
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%d %H:%M:%S')
        return super().default(obj)

### 7.2. Create the `list_into_chunks` function
This function splits a list into the chunks.

In [ ]:
def list_into_chunks(lst, chunk_size=100):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

### 7.3. Create the `string_into_chunks` function
This function splits a long text into the chunks.

In [ ]:
def string_into_chunks(string: str, max_tokens=TOKENS_LIMIT):
    if count_tokens(string) <= max_tokens:
        return [string]

    delimiter = ' '
    words = string.split(delimiter)
    chunks = []
    current_chunk = []

    for word in words:
        if count_tokens(delimiter.join(current_chunk + [word])) <= max_tokens:
            current_chunk.append(word)
        else:
            chunks.append(delimiter.join(current_chunk))
            current_chunk = [word]

    if current_chunk:
        chunks.append(delimiter.join(current_chunk))

    return chunks

### 7.4. Create the `clean_string` function
This function removes all html and markdown elements from a string. This function is required when the number of characters needs to be reduced before converting to embedding.

In [ ]:
def clean_string(string: str):
    def strip_html_elements(string: str):
        html = markdown(string)
        soup = BeautifulSoup(html, "html.parser")
        text = soup.get_text()
        return text.strip()

    def remove_unicode_escapes(string: str):
        return re.sub(r'[^\x00-\x7F]+', '', string)

    def remove_string_spaces(strgin: str):
        new_string = re.sub(r'\n+', '\n', strgin)
        new_string = re.sub(r'\s+', ' ', new_string)
        return new_string

    def remove_links(string: str):
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        return re.sub(url_pattern, '', string)

    new_string = strip_html_elements(string)
    new_string = remove_unicode_escapes(new_string)
    new_string = remove_string_spaces(new_string)
    new_string = re.sub(r'\*\*+', '*', new_string)
    new_string = re.sub(r'--+', '-', new_string)
    new_string = re.sub(r'====+', '=', new_string)
    new_string = remove_links(new_string)

    return new_string

## 8. Create Leaderboard functions

### 8.1. Create the `leaderboard_get_df` function
This function loads a pre-generated Open LLM Leaderboard dataset. Based on this dataset, all model data is created and inserted into the database.

In [ ]:
def leaderboard_get_df():
    response = requests.get(LEADERBOARD_DATASET_URL)

    if response.status_code == 200:
        data = json.loads(response.text)
        df = pd.DataFrame(data).head(MODELS_LIMIT)
        return df
    else:
        print("Failed to retrieve JSON file")

### 8.2. Create the `leaderboard_insert_model` function
This function prepares a model, creates embedding based on the LLM Leaderboard model data and inserts into the `models` and `model_readmes` tables.

In [ ]:
def leaderboard_insert_model(model):
    try:
        _model = {key: value for key, value in model.items() if key != 'readme'}
        to_embedding = json.dumps(_model, cls=JSONEncoder)
        embedding = str(create_embedding(to_embedding))
        model_to_insert = {**_model, embedding: embedding}
        readmes_to_insert = []

        if model['readme']:
            readme = {
                'model_repo_id': model['repo_id'],
                'text': model['readme'],
                'created_at': time()
            }

            if count_tokens(readme['text']) <= TOKENS_TRASHHOLD_LIMIT:
                readme['clean_text'] = clean_string(readme['text'])
                to_embedding = json.dumps({
                    'model_repo_id': readme['model_repo_id'],
                    'clean_text': readme['clean_text'],
                })
                readme['embedding'] = str(create_embedding(to_embedding))
                readmes_to_insert.append(readme)
            else:
                for i, chunk in enumerate(string_into_chunks(readme['text'])):
                    _readme = {
                        **readme,
                        'text': chunk,
                        'created_at': time()
                    }

                    _readme['clean_text'] = clean_string(chunk)
                    to_embedding = json.dumps({
                        'model_repo_id': _readme['model_repo_id'],
                        'clean_text': chunk,
                    })
                    _readme['embedding'] = str(create_embedding(to_embedding))
                    readmes_to_insert.append(_readme)

        with connection.cursor() as cursor:
            cursor.execute(f'''
                INSERT INTO {MODELS_TABLE_NAME} (name, author, repo_id, score, link, still_on_hub, arc, hellaswag, mmlu, truthfulqa, winogrande, gsm8k, downloads, likes, created_at, embedding)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, FROM_UNIXTIME(%s), JSON_ARRAY_PACK(%s))
            ''', tuple(model_to_insert.values()))

        for chunk in list_into_chunks([tuple(readme.values()) for readme in readmes_to_insert]):
            with connection.cursor() as cursor:
                cursor.executemany(f'''
                    INSERT INTO {MODEL_READMES_TABLE_NAME} (model_repo_id, text, created_at, clean_text, embedding)
                    VALUES (%s, %s, FROM_UNIXTIME(%s), %s, JSON_ARRAY_PACK(%s))
                ''', chunk)
    except Exception as e:
        print('Error leaderboard_insert_model: ', e)

### 8.3. Create the `leaderboard_process_models` function
This function retrieves model data from the LLM Leaderboard dataset that does not exist in the `models` table and inserts those models.

In [ ]:
def leaderboard_process_models():
    print('Processing models')

    existed_model_repo_ids = [i[0] for i in get_models('repo_id', as_dict=False)]
    leaderboard_df = leaderboard_get_df()

    for i, row in leaderboard_df.iterrows():
        if not row['repo_id'] in existed_model_repo_ids:
            leaderboard_insert_model(row.to_dict())

## 9. Create a GitHub client and functions

### 9.1. Create a GitHub client

In [ ]:
github = Github(auth=Auth.Token(GITHUB_ACCESS_TOKEN))

### 9.2. Create the `github_search_repos` function
This function search for GitHub repositories based on the keyword.

In [ ]:
def github_search_repos(keyword: str, last_created_at):
    repos = []
    query = f'"{keyword}" in:name,description,readme'

    if last_created_at:
        query += f' created:>{last_created_at}'

    try:
        for repo in github.search_repositories(query):
            try:
                readme_file = repo.get_readme()

                if readme_file.size > 7000:
                    continue

                readme = readme_file.decoded_content.decode('utf-8')

                repos.append({
                    'repo_id': repo.id,
                    'name': repo.name,
                    'link': repo.html_url,
                    'created_at': repo.created_at.timestamp(),
                    'description': repo.description if bool(repo.description) else '',
                    'readme': readme,
                })
            except Exception:
                continue
    except Exception:
        return repos

    return repos

### 9.3. Create the `github_insert_model_repos` function
This function prepares a repository, creates embedding based on the repository readme text and inserts into the `model_github_repos` table.

In [ ]:
def github_insert_model_repos(model_repo_id, repos):
    for repo in repos:
        try:
            values = []
            value = {
                'model_repo_id': model_repo_id,
                'repo_id': repo['repo_id'],
                'name': repo['name'],
                'description': repo['description'],
                'clean_text': clean_string(repo['readme']),
                'link': repo['link'],
                'created_at': repo['created_at'],
            }

            to_embedding = {
                'model_repo_id': model_repo_id,
                'name': value['name'],
                'description': value['description'],
                'clean_text': value['clean_text']
            }

            if count_tokens(value['clean_text']) <= TOKENS_TRASHHOLD_LIMIT:
                embedding = str(create_embedding(json.dumps(to_embedding)))
                values.append({**value, 'embedding': embedding})
            else:
                for chunk in string_into_chunks(value['clean_text']):
                    embedding = str(create_embedding(json.dumps({
                        **to_embedding,
                        'clean_text': chunk
                    })))
                    values.append({**value, 'clean_text': chunk, 'embedding': embedding})

            for chunk in list_into_chunks([list(value.values()) for value in values]):
                with connection.cursor() as cursor:
                    cursor.executemany(f'''
                        INSERT INTO {MODEL_GITHUB_REPOS_TABLE_NAME} (model_repo_id, repo_id, name, description, clean_text, link, created_at, embedding)
                        VALUES (%s, %s, %s, %s, %s, %s, FROM_UNIXTIME(%s), JSON_ARRAY_PACK(%s))
                    ''', chunk)
        except Exception as e:
            print('Error github_insert_model_repos: ', e)

### 9.4. Create the `github_process_models_repos` function
This function looks for all GitHub repositories that are newer than the latest `created_at` value from `model_github_repos` for each model, and inserts the found repositories into `model_github_repos`.

In [ ]:
def github_process_models_repos(existed_models):
    print('Processing GitHub posts')

    for model in existed_models:
        try:
            repo_id = model['repo_id']
            last_created_at = db_get_last_created_at(MODEL_GITHUB_REPOS_TABLE_NAME, repo_id, True)
            keyword = model['name'] if re.search(r'\d', model['name']) else repo_id
            found_repos = github_search_repos(keyword, last_created_at)

            if len(found_repos):
                github_insert_model_repos(repo_id, found_repos)
        except Exception as e:
            print('Error github_process_models_repos: ', e)

## 10. Create a Twitter client and functions

### 10.1. Create a Twitter client

In [ ]:
twitter = tweepy.Client(TWITTER_BEARER_TOKEN)

### 10.2. Create the `twitter_search_posts` function
This function search for Twitter posts based on the keyword.

In [ ]:
def twitter_search_posts(keyword, last_created_at):
    posts = []

    try:
        tweets = twitter.search_recent_tweets(
            query=f'{keyword} -is:retweet',
            tweet_fields=['id', 'text', 'created_at'],
            start_time=last_created_at,
            max_results=10
        )

        for tweet in tweets.data:
            posts.append({
                'post_id': tweet.id,
                'text': tweet.text,
                'created_at': tweet.created_at,
            })
    except Exception:
        return posts

    return posts

### 10.3. Create the `twitter_insert_model_posts` function
This function prepares a Twitter post, creates embedding based on the post text and inserts into the `model_twitter_posts` table.

In [ ]:
def twitter_insert_model_posts(model_repo_id, posts):
    for post in posts:
        try:
            values = []

            value = {
                'model_repo_id': model_repo_id,
                'post_id': post['post_id'],
                'clean_text': clean_string(post['text']),
                'created_at': post['created_at'],
            }

            to_embedding = {
                'model_repo_id': value['model_repo_id'],
                'clean_text': value['clean_text']
            }

            embedding = str(create_embedding(json.dumps(to_embedding)))
            values.append({**value, 'embedding': embedding})

            for chunk in list_into_chunks([list(value.values()) for value in values]):
                with connection.cursor() as cursor:
                    cursor.executemany(f'''
                        INSERT INTO {MODEL_TWITTER_POSTS_TABLE_NAME} (model_repo_id, post_id, clean_text, created_at, embedding)
                        VALUES (%s, %s, %s, FROM_UNIXTIME(%s), JSON_ARRAY_PACK(%s))
                    ''', chunk)
        except Exception as e:
            print('Error twitter_insert_model_posts: ', e)

### 10.4. Create the `twitter_process_models_posts` function
This function looks for all Twitter posts that are newer than the latest `created_at` value from `model_twitter_posts` for each model, and inserts the found posts into `model_twitter_posts`.

In [ ]:
def twitter_process_models_posts(existed_models):
    print('Processing Twitter posts')

    for model in existed_models:
        try:
            repo_id = model['repo_id']
            last_created_at = db_get_last_created_at(MODEL_TWITTER_POSTS_TABLE_NAME, repo_id, True)
            keyword = model['name'] if re.search(r'\d', model['name']) else repo_id
            found_posts = twitter_search_posts(keyword, last_created_at)

            if len(found_posts):
                twitter_insert_model_posts(repo_id, found_posts)
        except Exception as e:
            print('Error twitter_process_models_posts: ', e)

## 11. Create a Reddit client and functions

### 11.1. Create a Twitter client

In [ ]:
reddit = praw.Reddit(
    username=REDDIT_USERNAME,
    password=REDDIT_PASSWORD,
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

### 11.2. Create the `reddit_search_posts` function
This function search for Reddit posts based on the keyword.

In [ ]:
def reddit_search_posts(keyword: str, last_created_at):
    posts = []

    try:
        for post in reddit.subreddit('all').search(
            f'"{keyword}"', sort='relevance', time_filter='year', limit=100
        ):
            contains_keyword = keyword in post.title or keyword in post.selftext

            if contains_keyword and not post.over_18:
                if not last_created_at or (post.created_utc > last_created_at):
                    posts.append({
                        'post_id': post.id,
                        'title': post.title,
                        'text': post.selftext,
                        'link': f'https://www.reddit.com{post.permalink}',
                        'created_at': post.created_utc,
                    })
    except Exception as e:
        print('Error reddit_search_posts: ', e)
        return posts

    return posts

### 11.3. Create the `reddit_insert_model_posts` function
This function prepares a Reddit post, creates embedding based on the post text and inserts into the `model_reddit_posts` table.

In [ ]:
def reddit_insert_model_posts(model_repo_id, posts):
    for post in posts:
        try:
            values = []

            value = {
                'model_repo_id': model_repo_id,
                'post_id': post['post_id'],
                'title': post['title'],
                'clean_text': clean_string(post['text']),
                'link': post['link'],
                'created_at': post['created_at'],
            }

            to_embedding = {
                'model_repo_id': model_repo_id,
                'title': value['title'],
                'clean_text': value['clean_text']
            }

            if count_tokens(value['clean_text']) <= TOKENS_TRASHHOLD_LIMIT:
                embedding = str(create_embedding(json.dumps(to_embedding)))
                values.append({**value, 'embedding': embedding})
            else:
                for chunk in string_into_chunks(value['clean_text']):
                    embedding = str(create_embedding(json.dumps({
                        **to_embedding,
                        'clean_text': chunk
                    })))
                    values.append({**value, 'clean_text': chunk, 'embedding': embedding})

            for chunk in list_into_chunks([list(value.values()) for value in values]):
                with connection.cursor() as cursor:
                    cursor.executemany(f'''
                        INSERT INTO {MODEL_REDDIT_POSTS_TABLE_NAME} (model_repo_id, post_id, title, clean_text, link, created_at, embedding)
                        VALUES (%s, %s, %s, %s, %s, FROM_UNIXTIME(%s), JSON_ARRAY_PACK(%s))
                    ''', chunk)
        except Exception as e:
            print('Error reddit_insert_model_posts: ', e)

### 11.4. Create the `reddit_process_models_posts` function
This function looks for all Reddit posts that are newer than the latest `created_at` value from `model_reddit_posts` for each model, and inserts the found posts into `model_reddit_posts`.

In [ ]:
def reddit_process_models_posts(existed_models):
    print('Processing Reddit posts')

    for model in existed_models:
        try:
            repo_id = model['repo_id']
            last_created_at = db_get_last_created_at(MODEL_REDDIT_POSTS_TABLE_NAME, repo_id)
            keyword = model['name'] if re.search(r'\d', model['name']) else repo_id
            found_posts = reddit_search_posts(keyword, last_created_at)

            if len(found_posts):
                reddit_insert_model_posts(repo_id, found_posts)
        except Exception as e:
            print('Error reddit_process_models_posts: ', e)

## 12. Run the 
First, the notebook creates tables in the database if they don't exist.
Next, the notebook retrieves the specified number of models from the Open LLM Leaderboard dataset, creates embeddings, and enters the data into the `models` and `model_reamdes` tables.
Next, it executes a query to retrieve all the models in the database. Based on these models, Twitter posts, Reddit posts, and GitHub repositories are searched, converted into embeddings and inserted into tables.

Finally, we get a ready set of data for finding the most appropriate model for any use case using semantic search.

In [ ]:
create_tables()

leaderboard_process_models()

existed_models = get_models('repo_id, name', 'ORDER BY score DESC')

twitter_process_models_posts(existed_models)
reddit_process_models_posts(existed_models)
github_process_models_repos(existed_models)